In [3]:
import time
import sys
from pynq import Overlay, MMIO, allocate
from pynq import MMIO
import numpy as np
ol = Overlay("pulseAcq.bit")

class PulseAcq():
    def __init__(self,GPIO_ADDRESS,GPIO_RANGE,DMA_ADDRESS,DMA_RANGE):
        self.gpio = MMIO(GPIO_ADDRESS, GPIO_RANGE)
        self.dma = MMIO(DMA_ADDRESS, DMA_RANGE)
        self.gpioDataOut=0
        
    def setResetn(self,val):
        mask=0b1111111111111111111111110
        self.gpioDataOut=(self.gpioDataOut & mask) | (val << 0)
        self.gpio.write(0, self.gpioDataOut)
        return

    def setCounterMax(self,val):
        mask=0b0000000000000000000000001
        self.gpioDataOut=(self.gpioDataOut & mask) | (val << 1)
        self.gpio.write(0, self.gpioDataOut)
        return
    def getState(self):
        return self.gpio.read(0x0008) & 0b111
    
    def getStreamUpCounter(self):
        return (self.gpio.read(0x0008) >> 3)
        
    def dmaS2MMIsIdle(self):
        isIdle=bool(self.dma.read(0x34) & (1<<1))
        return isIdle
    
    def dmaS2MMConfig(self,bufferAddress):
        self.dma.write(0x48,bufferAddress)
        return
    
    def dmaS2MMRun(self,bufferBytesLen):
        self.dma.write(0x58,bufferBytesLen)
        return
    
    def dmaS2MMReset(self):
        self.dma.write(0x30,(1<<2))
        return
    
    def dmaS2MMHalt(self):
        self.dma.write(0x30,0)
        return 
    
    def dmaS2MMStart(self):
        self.dma.write(0x30,1)

In [9]:
####################################################
# Parameters
####################################################
# Physical addresses
GPIO_ADDRESS=0x41200000
GPIO_RANGE=0x10000
DMA_ADDRESS=0x40400000
DMA_RANGE=0x10000

#Integration window (in units of 10ns)
counterMax= 1000  

#Buffer length (in units of 64 bits)
bufferLen= 100 

####################################################
# Instantiations and memory allocations
####################################################
pulseAcq=PulseAcq(GPIO_ADDRESS,GPIO_RANGE,DMA_ADDRESS,DMA_RANGE)      
bufferType=np.uint64

buffer = allocate(bufferLen, dtype=bufferType)
bufferAddress=buffer.physical_address
bufferBytesLen=buffer.nbytes

####################################################
# Initial config
####################################################
pulseAcq.setCounterMax(counterMax)
pulseAcq.dmaS2MMHalt()
pulseAcq.dmaS2MMReset()
pulseAcq.dmaS2MMConfig(bufferAddress)
pulseAcq.dmaS2MMStart()

####################################################
#  Run pulse acquisition
####################################################
pulseAcq.setResetn(1)                        #Enable pulse acquisition core
pulseAcq.dmaS2MMRun(bufferBytesLen)          #Run DMA engine
while(not(pulseAcq.dmaS2MMIsIdle())): pass    #Wait for pulse acquisition to finsih
print("Pulse acquisition finished.\nStream up counter: {}\nCurrent state: {} (0->off 1->idle 2->run 4->error)".format(pulseAcq.getStreamUpCounter(),pulseAcq.getState()))

####################################################
#  Data processing
####################################################
counterList=[]
timestampList=[]
maskList=[]

for i in range(0,pulseAcq.getStreamUpCounter()):
    counterList+= [(int(buffer[i])>>0) & 0xffffff]
    timestampList+= [(int(buffer[i])>>24) & 0xff]
    maskList+= [(int(buffer[i])>>32) & 0xffffffff]
print("\nDATA RX:")
print("counterList: {}\ntimestampList: {}\nmaskList: {}".format(counterList,timestampList,maskList))


Pulse acquisition finished.
Stream up counter: 7
Current state: 1 (0->off 1->idle 2->run 4->error)

DATA RX:
counterList: [5, 12, 13, 22, 27, 104, 112]
timestampList: [99, 99, 99, 99, 99, 99, 99]
maskList: [1, 2, 1, 7, 4, 2, 255]
